In [1]:
%%capture
!pip install git+https://github.com/vyperlang/titanoboa.git@master

In [2]:
from google.colab import userdata
import boa

ModuleNotFoundError: No module named 'google'

## **Setting Up the Infrastructure**

Before we can simulate some transactions, we need to initialize the necessary contract. For API keys, we can use the "Secrets" feature of Google Colab. An `RPC_ETHEREUM` including an HTTP key (e.g., from Alchemy) and an `ETHERSCAN_API_KEY` including an Etherscan API key need to be created before the notebook can be used.

Interacting with the contract is very simple when using titanoboa. The `from_etherscan` function lets us connect to the contracts.

`boa.env.fork` forks the desired chain for us.

Official Titanoboa Documentation: [https://titanoboa.readthedocs.io/en/latest/](https://titanoboa.readthedocs.io/en/latest/)


In [ ]:
boa.env.fork(userdata.get('RPC_ETHEREUM'))

VAULT = '0xcea18a8752bb7e7817f9ae7565328fe415c0f2ca'
CRVUSD = '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'

vault = boa.from_etherscan(
    VAULT,
    name='crvUSD Vault for CRV Lending Market',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

crvusd = boa.from_etherscan(
    CRVUSD,
    name='crvUSD',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

vault, crvusd

---

## **Obtaining Shares**

There are two functions that let users mint vault shares directly from the vault. Because these shares are tradable, they can also be obtained in other ways.

_However, shares can only be **minted** from the Vault contract using these two functions:_

- `deposit`: Lets a user deposit a number of assets and receive the corresponding number of shares in exchange.
- `mint`: Lets a user mint an exact number of shares by depositing only the necessary amount of assets.

---

_Before being able to deposit assets and mint shares, the user needs to grant approval to the contract to spend the assets:_


In [ ]:
USER = '0xcfB8F97F1602C3a8Fa91D19f62C069457e1A8c37' #random user with crvUSD and ETH for gas

with boa.env.prank(USER):
  crvusd.approve(vault, 2**256-1)
  assert crvusd.allowance(USER, VAULT) == 2**256-1

print("Amount approved for the Vault to spend:", crvusd.allowance(USER, VAULT))

---

### **`deposit`**

`Vault.deposit(assets: uint256, receiver: address = msg.sender) -> uint256:`

Function to deposit a specified number of assets of the underlying token (`borrowed_token`) into the vault and mint the corresponding amount of shares to `receiver`.

Returns: minted shares (`uint256`).

| Input      | Type      | Description                                    |
|------------|-----------|------------------------------------------------|
| `assets`   | `uint256` | Amount of assets to deposit.                   |
| `receiver` | `address` | Receiver of the minted shares. Defaults to `msg.sender`. |


In [ ]:
def deposit_assets(assets, receiver):

  crvusd_balance_before = crvusd.balanceOf(USER)
  share_balance_before = vault.balanceOf(USER)
  print("crvusd balance before deposit", round(crvusd_balance_before / 1e18, 2))
  print("share balance before deopsit", round(share_balance_before / 1e18, 2))

  with boa.env.prank(USER):
    # deposit assets into the vault
    vault.deposit(assets, receiver)

  crvusd_balance_after = crvusd.balanceOf(USER)
  share_balance_after = vault.balanceOf(USER)
  print("crvusd balance after deposit", round(crvusd_balance_after / 1e18, 2))
  print("share balance after deopsit", round(share_balance_after / 1e18, 2))

  print("--------------------------")
  print("crvusd difference:", round((crvusd_balance_after - crvusd_balance_before) / 1e18, 2))
  print("share difference:", round((share_balance_after - share_balance_before) / 1e18, 2))

assets_to_deposit = 1000 * 10**18
shares_receiver = USER
deposit_assets(assets_to_deposit, shares_receiver)

_There are some additional "helper functions" for depositing assets:_

- `maxDeposit`: This function returns the maximum amount of assets a user can deposit into the vault, which is essentially the asset balance of the user.
- `previewDeposit`: This function simulates the `deposit` function and returns the number of shares the user ends up with when depositing a specific number of assets.

---

### **`mint`**

`Vault.mint(shares: uint256, receiver: address = msg.sender) -> uint256:`

Function to mint a specific amount of shares (`shares`) to `receiver` by depositing the necessary number of assets into the vault.

Returns: amount of assets deposited (`uint256`).

| Input      | Type      | Description                                    |
|------------|-----------|------------------------------------------------|
| `shares`   | `uint256` | Number of shares to be minted.                 |
| `receiver` | `address` | Receiver of the minted shares. Defaults to `msg.sender`. |


In [ ]:
def mint_shares(shares, receiver):

  crvusd_balance_before = crvusd.balanceOf(USER)
  share_balance_before = vault.balanceOf(USER)
  print("crvusd balance before deposit", round(crvusd_balance_before / 1e18, 2))
  print("share balance before deopsit", round(share_balance_before / 1e18, 2))

  with boa.env.prank(USER):
    # deposit assets into the vault
    vault.mint(shares, receiver)

  crvusd_balance_after = crvusd.balanceOf(USER)
  share_balance_after = vault.balanceOf(USER)
  print("crvusd balance after deposit", round(crvusd_balance_after / 1e18, 2))
  print("share balance after deopsit", round(share_balance_after / 1e18, 2))

  print("--------------------------")
  print("crvusd difference:", round((crvusd_balance_after - crvusd_balance_before) / 1e18, 2))
  print("share difference:", round((share_balance_after - share_balance_before) / 1e18, 2))

shares_to_mint = 100000 * 10**18
shares_receiver = USER
mint_shares(shares_to_mint, shares_receiver)

_There are some additional "helper functions" for depositing assets:_

- `maxMint`: This function returns the maximum number of mintable shares, which is essentially `max_value(uint256)`.
- `previewMint`: This function simulates the `mint` function and returns the number of assets required to mint the shares.
